In [2]:
! pip install datasets

In [3]:
from datasets import load_dataset
import numpy as np
import wave
import os 

In [4]:
UA = load_dataset("ngdiana/uaspeech_severity_high")

In [5]:
UA_df = UA['train'].to_pandas()

In [6]:
UA_df = UA_df

In [7]:
UA_df['filename'] = UA_df['path'].apply(lambda x: x.split("/")[-1])



In [8]:
import pandas as pd
import os 

In [9]:
UA_df.to_csv("UA_df.csv")

In [10]:
def array2WAV(id):
    
    # Define the sample rate and number of samples
    sample_rate = 16000
    num_samples = 1
    
    # Create a WAV file object
    wav_file = wave.open(UA_df['filename'].iloc[id], "w")
    
    # Set the WAV file parameters
    wav_file.setnchannels(1) # 1 channel (mono)
    wav_file.setsampwidth(2) # 16-bit sample width
    wav_file.setframerate(sample_rate)
    
    # Write the samples to the WAV file as binary data
    
    samples = UA_df['speech'].iloc[id]
    samples = (samples * (2**15 - 1)).astype(np.int16)
    wav_file.writeframes(samples.tobytes())
    
    # Close the WAV file
    wav_file.close()

# set the output directory to save the wav files
out_dir = r'/teamspace/studios/this_studio/output'

os.chdir(out_dir)

for id in range(0, len(UA_df)):
    array2WAV(id)

In [2]:
!  pip install huggingsound

  Using cached huggingsound-0.1.6-py3-none-any.whl.metadata (7.1 kB)
  Using cached jiwer-2.6.0-py3-none-any.whl.metadata (14 kB)
  Using cached librosa-0.9.2-py3-none-any.whl.metadata (8.2 kB)
  Using cached torch-1.12.1-cp310-cp310-manylinux1_x86_64.whl.metadata (22 kB)
  Using cached transformers-4.41.2-py3-none-any.whl.metadata (43 kB)
  Using cached rapidfuzz-2.13.7-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.0 kB)
  Using cached audioread-3.0.1-py3-none-any.whl.metadata (8.4 kB)
  Using cached resampy-0.4.3-py3-none-any.whl.metadata (3.0 kB)
  Using cached numba-0.59.1-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (2.7 kB)
  Using cached soundfile-0.12.1-py2.py3-none-manylinux_2_31_x86_64.whl.metadata (14 kB)
  Using cached pooch-1.8.2-py3-none-any.whl.metadata (10 kB)
  Using cached regex-2024.5.15-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
  Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_1

In [6]:
! pip install accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 24.1 MB/s eta 0:00:00


In [1]:
from huggingsound import TrainingArguments, ModelArguments, SpeechRecognitionModel, TokenSet
#https://github.com/jonatasgrosman/huggingsound
#!pip install huggingsound
import pandas as pd
import os


audio_dir = r'/teamspace/studios/this_studio/output'

import torch
torch.cuda.empty_cache()
#device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
#model = SpeechRecognitionModel("facebook/wav2vec2-large-xlsr-53")
model = SpeechRecognitionModel("facebook/wav2vec2-large-xlsr-53", device=device)
torch.cuda.empty_cache()

# preparing the vocab file
vocab_dict = {'c': 0,
 'q': 1,
 'w': 2,
 'j': 3,
 'r': 4,
 'h': 5,
 'x': 6,
 'm': 7,
 'p': 8,
 'd': 9,
 'f': 10,
 'g': 11,
 'k': 12,
 'u': 13,
 'v': 14,
 'a': 15,
 'n': 16,
 ' ': 17,
 'i': 18,
 's': 19,
 'y': 20,
 'l': 21,
 'e': 22,
 'o': 23,
 'z': 24,
 'b': 25,
 't': 26}

vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)

tokens = list(vocab_dict.keys())
token_set = TokenSet(tokens)

#%%
training_args = TrainingArguments(
    learning_rate=3e-4,
    max_steps=1000,
    eval_steps=200,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
)
model_args = ModelArguments(
    activation_dropout=0.1,
    hidden_dropout=0.1,
) 

#%%
# preparing the training data
UA_df = pd.read_csv(r'/teamspace/studios/this_studio/UA_df.csv')

os.chdir(audio_dir)

train_data = []
for id in range(0, len(UA_df)):
    train_data.append({"path": UA_df['filename'].iloc[id], "transcription": UA_df['target'].iloc[id]})

# for evaluation data    
UA_df[['col1', 'col2', 'col3', 'col4']] = UA_df['filename'].str.split("_", expand=True)
UA_df1 = UA_df.drop_duplicates(subset='col3', keep="first")

eval_data = []
for id in range(0, len(UA_df1)):
    eval_data.append({"path": UA_df1['filename'].iloc[id], "transcription": UA_df1['target'].iloc[id]})

# and finally, fine-tune your model
output_dir = r'/teamspace/studios/this_studio/models'
model.finetune(
    output_dir, 
    train_data=train_data, 
    eval_data=eval_data, # the eval_data is optional
    token_set=token_set,
    training_args=training_args,
    model_args=model_args,
)
#%%

2024-06-10 14:40:25.988193: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-10 14:40:27.347095: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


06/10/2024 14:40:28 - INFO - huggingsound.speech_recognition.model - Loading model...


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:54: FutureWarning: Loading a tokenizer inside Wav2Vec2Processor from a config that does not include a `tokenizer_class` attribute is deprecated and will be removed in v5. Please add `'tokenizer_class': 'Wav2Vec2CTCTokenizer'` attribute to either your `config.json` or `tokenizer_confi

06/10/2024 14:40:34 - WARNING - huggingsound.speech_recognition.model - Not fine-tuned model! You'll need to fine-tune it before use this model for audio transcription
06/10/2024 14:40:34 - WARNING - root - blank_token <pad> not in provided tokens. It will be added to the list of tokens
06/10/2024 14:40:34 - WARNING - root - unk_token <unk> not in provided tokens. It will be added to the list of tokens
06/10/2024 14:40:34 - WARNING - root - bos_token <s> not in provided tokens. It will be added to the list of tokens
06/10/2024 14:40:34 - WARNING - root - eos_token </s> not in provided tokens. It will be added to the list of tokens


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
`use_fast` is set to `True` but the tokenizer class does not have a fast version.  Falling back to the slow version.


...

06/10/2024 14:51:36 - INFO - huggingsound.trainer - Building trainer...


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:2274: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


06/10/2024 14:51:36 - INFO - huggingsound.trainer - Starting training...


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


Step,Training Loss,Validation Loss


In [ ]:
from huggingsound import SpeechRecognitionModel

model_dir = r'xx\xxx\xxx'

model = SpeechRecognitionModel(model_dir)

audio_dir =  r'xx\xxx\xxx'

import os
os.chdir(audio_dir)

audio_paths = ["F02_B1_C17_M6.wav"]

transcriptions = model.transcribe(audio_paths)